In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import numpy as np
import random
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, Dense
from keras.models import Model

# Instructions to run this code

To run it from your machine/gdrive, please change path to the location of this .ipynb file, and store the following files generated by `preprocessing.ipynb` in the same folder:
- x_train.npy
- y_train.npy
- x_test.npy
- y_test.npy

In [3]:
path = '/content/gdrive/My Drive/Colab Notebooks/EE627A Term Project/Ravdess/' # change this path according to your needs.

In [4]:
X_train = np.load(path+'x_train.npy')
X_test = np.load(path+'x_test.npy')

y_train = np.load(path+'y_train.npy')
y_test = np.load(path+'y_test.npy')

In [5]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [6]:
def clients(speech_list, labels, clients_number, intl='client'):
    client_name = ['{}_{}'.format(intl, i+1) for i in range(clients_number)]
    data = list(zip(speech_list, labels))
    size = len(data)//clients_number
    data_segments = [data[i:i + size] for i in range(0, size*clients_number, size)]
    assert(len(data_segments) == len(client_name))
    return {client_name[i] : data_segments[i] for i in range(len(client_name))} 

In [7]:
num_K = 10
clients = clients(x_traincnn, y_train, num_K, intl='client')

In [8]:
def batch_data(data_segments, batch_size):
    data, label = zip(*data_segments)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(batch_size)

In [9]:
B = 20
batch_client = dict()
for (client_name, data) in clients.items():
    batch_client[client_name] = batch_data(data,B)
    
test_batched = tf.data.Dataset.from_tensor_slices((x_traincnn, y_train)).batch(len(y_test))

# Convolutional Neural Network for Speech Emotion Recognition

In [10]:
class SER_CNN:
    @staticmethod
    def build(x_traincnn, y_train):
        model = Sequential()
        model.add(Conv1D(256, 5,padding='same', input_shape=(x_traincnn.shape[1],1), name = 'Conv1D_1', activation = 'relu'))
        model.add(Conv1D(128, 5,padding='same',name='Conv1D_2',activation='relu'))
        model.add(Dropout(0.1,name = 'Dropout_1'))
        model.add(MaxPooling1D(pool_size=(8), name = 'MaxPooling1D_1'))
        model.add(Conv1D(128, 5,padding='same',activation='relu',name = 'Conv1D_3'))
        model.add(Conv1D(128, 5, padding="valid",activation='relu',name='Conv1D_4'))
        model.add(Flatten(name = 'Flatten_1'))
        model.add(Dense(4,activation ='softmax',name= 'Dense_1'))
        return model

In [11]:
comms_round = 200
loss='categorical_crossentropy'
metrics = ['accuracy',tf.keras.metrics.Recall(),tf.keras.metrics.Precision() ]
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001, decay=1e-6)

In [12]:
def scaling_factor(clients_data, client): 
    name_client = list(clients_data.keys())
    bs = list(clients_data[client])[0][0].shape[0]
    count_global = sum([tf.data.experimental.cardinality(clients_data[client]).numpy() for client in name_client])*bs
    count_local = tf.data.experimental.cardinality(clients_data[client]).numpy()*bs
    return count_local/count_global

In [13]:
def model_weights(wght, var):

    weight = []
    step = len(wght)
    for i in range(step):
        weight.append(var * wght[i])
    return weight

In [14]:
def sum_weights(weight_list):
    
    avg = list()
    for tuple_list in zip(*weight_list):
        mean = tf.math.reduce_sum(tuple_list, axis=0)
        avg.append(mean)   
    return avg

In [15]:
def test_model(X_test, Y_test,  model, comm_round):

    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=32)
    loss = cce(Y_test, logits)
    acc =  accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    m = tf.keras.metrics.Recall()
    m.update_state(Y_test, logits)
    recall = m.result()
    return acc, loss, recall.numpy()

In [16]:
SER_CNN_global = SER_CNN()
global_model = SER_CNN_global.build(x_traincnn, y_train)
E = 5
global_model.summary()

history_acc = []
history_loss = []
history_recall = []
    

for comm_round in range(comms_round):
            

    global_weights = global_model.get_weights()
    
    
    scaled_local_weight_list = list()

    
    client_names= list(batch_client.keys())
    random.shuffle(client_names)
    
    
    for client in client_names:
        SER_CNN_local = SER_CNN()
        local_model = SER_CNN_local.build(x_traincnn, y_train)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        
        local_model.set_weights(global_weights)
        
        
        local_model.fit(batch_client[client], epochs=E, verbose=0) 
        
        
        scaling_factor_obj = scaling_factor(batch_client, client)
        scaled_weights = model_weights(local_model.get_weights(), scaling_factor_obj)
        scaled_local_weight_list.append(scaled_weights)
        
        
        K.clear_session()
        
    
    average_weights = sum_weights(scaled_local_weight_list)
    
    
    global_model.set_weights(average_weights)

    
    average_acc = 0
    average_loss = 0
    average_recall = 0
    count = 0

    for(x_testcnn, y_test) in test_batched:
        global_acc, global_loss, global_recall = test_model(x_testcnn, y_test, global_model, comm_round)
        average_acc += global_acc
        average_loss += global_loss
        average_recall += global_recall
        count+=1
    average_acc = average_acc/count
    average_recall = average_recall/count
    average_loss = average_loss/count
    print('comm_round: {} | global_acc: {:.3%} | global_recall {:.3%} | global_loss: {}'.format(comm_round, average_acc, average_recall, average_loss))
    history_acc.append(average_acc)
    history_loss.append(average_loss)
    history_recall.append(average_recall)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1D_1 (Conv1D)           (None, 97, 256)           1536      
                                                                 
 Conv1D_2 (Conv1D)           (None, 97, 128)           163968    
                                                                 
 Dropout_1 (Dropout)         (None, 97, 128)           0         
                                                                 
 MaxPooling1D_1 (MaxPooling1  (None, 12, 128)          0         
 D)                                                              
                                                                 
 Conv1D_3 (Conv1D)           (None, 12, 128)           82048     
                                                                 
 Conv1D_4 (Conv1D)           (None, 8, 128)            82048     
                                                        

# Saving the results
To save the results for a particular configuration change the file name (by changing the * with a number) below. The results will get stored as follows:
- history_acc*.npy - stores the training accuracy
- history_recall*.npy - stores the training recall
- history_loss*.npy - stores the training loss


The test configurations and the corresponding file numbers are as shown in the table below - 

|Sl. No. | Clients No.(K) | Batch Size(B) | Epochs(E) | File No.|
|--------|:---------------|:--------------|:----------|:--------|
|    1   |        5       |     10        |    1      |    1    |
|    2   |        5       |      10       |     5     |     2   |
|    3   |        5       |       20      |      1    |      3  |
|    4   |        5       |        20     |       5   |       4 |
|    5   |        10      |       10      | 1         |5        |
|    6   |        10      |         10    |  5        | 6       |
|    7   |        10      |        20     |   1       |  7      |
|    8   |        10      |        20     |    5      |   8     |

In [17]:

np.save(path+'/Results/history_acc5.npy',np.array(history_acc))
np.save(path+'/Results/history_recall5.npy',np.array(history_recall))
np.save(path+'/Results/history_loss5.npy',np.array(history_loss))